## Kate Groschner
## AY250 Homework 3

# 1) Monty: The Python Siri

Let's make a Siri-like program (call it Monty!) with the following properties:
   - record your voice command
   - use a webservice to parse that sound file into text
   - based on what the text, take three different types of actions:
       - send an email to yourself
       - do some math
       - tell a joke

So for example, if you say "Monty: email me with subject hello and body goodbye", it will email you with the appropriate subject and body. If you say "Monty: tell me a joke" then it will go to the web and find a joke and print it for you. If you say, "Monty: calculate two times three" it should response with printing the number 6.

Hint: you can use speed-to-text apps like Houndify (or, e.g., Google Speech https://cloud.google.com/speech/) to return the text (but not do the actions). You'll need to sign up for a free API and then follow documentation instructions for using the service within Python. 

# Question 1
My strategy for this question is to use the audio recording script Prof. Bloom showed in class. Send this to a webservice to parse, this will probably be done using request library to interact between the two. Taking the output text will be put into a list format if necessary in order to use python builtins to look at if the text contains certain words. This part of the script will most likely be a decision tree that leads to the last step that either a) uses stmp to send an email or b) does the math (which will also have a tree of if statements to figure out order of operations etc), or finally c) use request/soup to go to a website and get jokes

In [115]:
#Audio recording portion taken from lecture notes and cross referenced with the PyAudio example documentation
import pyaudio 
import wave

chunk = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "test1.wav"
p = pyaudio.PyAudio()
stream = p.open(format = FORMAT,
    channels = CHANNELS,
    rate = RATE,
    input = True,
    frames_per_buffer = chunk)
print("* recording")
frames = []
for i in range(0, int(RATE / chunk * RECORD_SECONDS)):
    data = stream.read(chunk)
    frames.append(data)
print("* done recording")
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

* recording
* done recording


In [116]:
#do the speech to text inquiry
import houndify
import wave
import time

CLIENT_ID = "yfAFR0XPoFNY4IRJh6RJJg=="
CLIENT_KEY = "sFhkKk6xA3tGW_Cl2VjM_FYI-guJmDwiCJonbBpiUa1LDVem1rA02pAZApejN1wxc4zpOLjLtZj43OA4ZdZ0Eg=="
AUDIO_FILE = "test1.wav"
BUFFER_SIZE = 512

class MyListener(houndify.HoundListener):
  def onPartialTranscript(self, transcript):
#     print("Partial transcript: " + transcript)
      return 
  def onFinalResponse(self, response):
    print("Final response: " + str(response))
  def onError(self, err):
    print("Error: " + str(err))

client = houndify.StreamingHoundClient(CLIENT_ID, CLIENT_KEY, "test_user")
audio = wave.open(AUDIO_FILE)
if audio.getsampwidth() != 2:
  print("%s: wrong sample width (must be 16-bit)" % AUDIO_FILE)
if audio.getframerate() != 8000 and audio.getframerate() != 16000:
  print("%s: unsupported sampling frequency (must be either 8 or 16 khz)" % AUDIO_FILE)
if audio.getnchannels() != 1:
  print("%s: must be single channel (mono)" % AUDIO_FILE)

client.setSampleRate(audio.getframerate())
client.start(MyListener())

while True:
  samples = audio.readframes(BUFFER_SIZE)
  if len(samples) == 0: break
  if client.fill(samples): break
  
       
result = client.finish() # returns either final response or error


Final response: {'Format': 'SoundHoundVoiceSearchResult', 'FormatVersion': '1.0', 'Status': 'OK', 'NumToReturn': 1, 'AllResults': [{'CommandKind': 'SpeechToTextOnlyCommand', 'SpokenResponse': 'manti what is two plus two', 'SpokenResponseLong': 'manti what is two plus two', 'WrittenResponse': 'Manti what is two plus two', 'WrittenResponseLong': 'Manti what is two plus two', 'AutoListen': False, 'ViewType': ['Native', 'None'], 'RawTranscription': 'manti what is two plus two', 'FormattedTranscription': 'Manti what is two plus two', 'NativeData': {'RawTranscription': 'manti what is two plus two', 'FormattedTranscription': 'Manti what is two plus two'}}], 'Disambiguation': {'NumToShow': 1, 'ChoiceData': [{'Transcription': 'manti what is two plus two', 'ConfidenceScore': 1, 'FixedTranscription': 'manti what is two plus two'}]}, 'ResultsAreFinal': [True], 'DomainUsage': [{'Domain': 'Speech To Text Only', 'DomainUniqueID': '3224afd8-821b-4e3e-b206-d92d160048cb', 'CreditsUsed': 0}], 'BuildInfo'

In [117]:
result['AllResults'][0]['FormattedTranscription']

'Manti what is two plus two'

In [ ]:
#Here I define a math function to call to handle requests to do math
def monty_math(text_in):
    

In [76]:
#Portion of the code to grab jokes from a website.
from urllib.request import urlopen
from bs4 import BeautifulSoup
import json
jk_url = 'http://pun.me/pages/dad-jokes.php'
response = urlopen(jk_url)
jk = response.read().decode('-utf-8')
response.close()
jk_soup = BeautifulSoup(jk,"html.parser")
items = jk_soup.findAll("li")
#now that I have a list of all the content items from the page I want to strip the html off
#and the items that are buttons to other pages and just have strings
jokes  = [i.text.strip() for i in items if "<a href=" not in str(i)]
jokes

['Did you hear about the restaurant on the moon? Great food, no atmosphere.',
 'What do you call a fake noodle? An Impasta.',
 'How many apples grow on a tree? All of them.',
 "Want to hear a joke about paper? Nevermind it's tearable.",
 "I just watched a program about beavers. It was the best dam program I've ever seen.",
 'Why did the coffee file a police report? It got mugged.',
 "How does a penguin build it's house? Igloos it together.",
 'Dad, did you get a haircut? No I got them all cut.',
 'What do you call a Mexican who has lost his car? Carlos.',
 "Dad, can you put my shoes on? No, I don't think they'll fit me.",
 'Why did the scarecrow win an award? Because he was outstanding in his field.',
 "Why don't skeletons ever go trick or treating? Because they have no body to go with.",
 "Ill call you later. Don't call me later, call me Dad.",
 "What do you call an elephant that doesn't matter? An irrelephant",
 "Want to hear a joke about construction? I'm still working on it.",
 "Wh

# 2) Write a program that identifies musical notes from sound (AIFF) files. 

  - Run it on the supplied sound files (12) and report your program’s results. 
  - Use the labeled sounds (4) to make sure it works correctly. The provided sound files contain 1-3 simultaneous notes from different organs.
  - Save copies of any example plots to illustrate how your program works.
  
  https://piazza.com/berkeley/spring2018/ay250class13410/resources -> Homeworks -> hw3_sound_files.zip

Hints: You’ll want to decompose the sound into a frequency power spectrum. Use a Fast Fourier Transform. Be care about “unpacking” the string hexcode into python data structures. The sound files use 32 bit data. Play around with what happens when you convert the string data to other integer sizes, or signed vs unsigned integers. Also, beware of harmonics.